In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
# pd.set_option('max_columns', 200)         # need to find the new way to say this

     Agilent RT (min)
     
H2 peak at: 4.28

O2 peak at:	4.61

N2 peak at:	4.95
	
H2 peak at:	0.88
(SRI)	

In [ ]:
# define which columns to import for each sheet in the file
# channel 4 is Ar/SRI H2, channel 2 is N2/Agilent O2
# using Agilent alone, channel 2 is N2, and corresponds to post-GC mass flow rate
flowrates_col_list = ['Timestamp_FR', 'mol/s_2']   # timestamp is same between channels in this case
O2_GC_col_list = ['Timestamp_O2', 'RT [min]_O2', 'Area_O2', 'Height_O2']
#O2XO_GC_col_list = ['Timestamp_O2XO', 'RT [min]_O2XO', 'Area_O2XO', 'Height_O2XO']
H2_GC_col_list = ['Timestamp_H2', 'RT [min]_H2', 'Area_H2', 'Height_H2']
#H2XO_GC_col_list = ['Timestamp_H2XO', 'RT [min]_H2XO', 'Area_H2XO', 'Height_H2XO']
CP_col_list = ['Timestamp_CP', 'Ewe/V', 'I/mA']

In [ ]:
# import each sheet with the proper header row defined
# Insert for header the Excel sheet row number - 1, so for headers in row 66, insert 65.
df_CP = pd.read_excel('Pt-C-1_CP.xlsx', sheet_name="Sheet1", header=[61], usecols = CP_col_list)
df_FR = pd.read_excel('Pt-C-1_FR.xlsx', sheet_name="Sheet1", header=[0],
                     usecols=flowrates_col_list)
df_O2GC = pd.read_excel('Pt-C-1_GC.xlsx', sheet_name='O2', header=[0], usecols = O2_GC_col_list)
#df_O2XOGC = pd.read_excel('Si-GC-H2-O2.xlsx', sheet_name='O2XO', header=[0], usecols = O2XO_GC_col_list)
df_H2GC = pd.read_excel('Pt-C-1_GC.xlsx', sheet_name='H2', header=[0], usecols = H2_GC_col_list)
#df_H2XOGC = pd.read_excel('Si-GC-H2-O2.xlsx', sheet_name='H2XO', header=[0], usecols = H2XO_GC_col_list)

df_H2GC.head()

In [ ]:
df_CP.head()

In [ ]:
# rename columns before merge--will merge by the timestamp column

df_H2GC.rename(columns={'Timestamp_H2':'timestamp'}, inplace=True)
#df_H2XOGC.rename(columns={'Timestamp_H2XO':'timestamp'}, inplace=True)
df_O2GC.rename(columns={'Timestamp_O2':'timestamp'}, inplace=True)
#df_O2XOGC.rename(columns={'Timestamp_O2XO':'timestamp'}, inplace=True)
df_CP.rename(columns={'Timestamp_CP':'timestamp'}, inplace=True)
df_FR.rename(columns={'Timestamp_FR':'timestamp'}, inplace=True)

df_O2GC.head()

In [ ]:
df_CP.head()

In [ ]:
df_O2GC.head()

In [ ]:
# Here is where the magic happens, and the different dataframes are concatenated
# and a single timestamp column is produced with all values.

# make a list of the dataframes to merge
#df_all_list = [df_CP, df_O2GC]
df_all_list = [df_CP, df_FR, df_H2GC, df_O2GC] #, df_H2XOGC, df_O2XOGC]

# concatenate along axis 0, pasting the GC data at the end of the flowrate and CP data
df_all = pd.concat(df_all_list, axis=0, ignore_index=True)

# sort rows by timestamp, which locates the peaks from the GC properly with respect to the current and flowrate measurements
df_all.sort_values(by='timestamp', ascending=True, inplace=True)
df_all

In [ ]:
len_column = df_all['timestamp'].shape[0]
#print(len_column)

In [ ]:
# add columns for average I and avg mol/s
# create series to average over 5 surrounding current values
# create series to average over 5 surrounding flow rate values
# place those series below when done

H2_slope = 135.068     # Agilent 2sccm flow rate
H2_intercept = 0          # Agilent 2sccm flow rate
#H2_slope = 130.823091     # Agilent 5sccm flow rate
#H2_intercept = 0          # Agilent 5sccm flow rate
#H2_slope = 121.27     # Agilent June 2021 10sccm flow rate
#H2_intercept = 7.2112   # Agilent June 2021
#H2_slope = 18.544          # SRI
#H2_intercept = -8.6855     # SRI
O2_slope = 885
O2_intercept = -730
#O2_slope = 850.876      # Agilent new
#O2_intercept = -377.989      # Agilent new calibration


# add columns to calculate ppm, mol/s product, and FE for main H2 and O2 peaks
# currently just at calculating ppm from the calibration curves
# df.insert(2, 'new-col', data)
df_all.insert(3, 'I_avg(A)', np.zeros(len_column))
df_all.insert(5, 'mol/s_avg', np.zeros(len_column))
# Calculate H2 ppm by multiplying H2 area by calibration value for Agilent
# SRI has slope and intercept for H2
df_all.insert(9, 'H2_ppm', df_all['Area_H2']*H2_slope+H2_intercept)
#df_all.insert(10, 'H2_mol/s', df_all['H2_ppm']*df_all['mol/s_2']*0.000001)
df_all.insert(10, 'H2_mol/s', np.zeros(len_column))
df_all.insert(11, 'H2_FE', np.zeros(len_column))
# Calculate O2 ppm by using O2 area calibration curve for Agilent
df_all.insert(15, 'O2_ppm', df_all['Area_O2']*O2_slope+O2_intercept)
#df_all.insert(15, 'O2_ppm', np.zeros(len_column))
df_all.insert(16, 'O2_mol/s', np.zeros(len_column))
df_all.insert(17, 'O2_FE', np.zeros(len_column))
df_all

In [ ]:
# add columns to calculate ppm for crossover H2 and O2 peaks
# Calculate H2, O2 ppm by using the calibration for the Agilent, for O2 in the H2 channel
#df_all.insert(21, 'H2XO_ppm', df_all['Area_H2XO']*H2_slope+H2_intercept)
#df_all.insert(22, 'H2XO_mol/s', np.zeros(len_column))
#df_all.insert(23, 'H2XO_FE', np.zeros(len_column))
#df_all.insert(27, 'O2XO_ppm', df_all['Area_O2XO']*O2_slope+O2_intercept)
#df_all.insert(28, 'O2XO_mol/s', np.zeros(len_column))
#df_all.insert(29, 'O2XO_FE', np.zeros(len_column))

#df_all

In [ ]:
# plot each variable
#cols_y = list(df_all.columns)[1:]
# plot just some variables
cols_y = (df_all['Area_H2'], df_all['Area_O2'], df_all['I/mA']) #df_all['Area_O2XO'], df_all['Area_H2XO'], )
for y in cols_y:
    fig_ = px.scatter(df_all, x='timestamp', y=y)
    fig_.show()
    
#print("Area_H2 and Area_O2")
#fig_ = px.scatter(df_all, x='timestamp', y=['Area_O2', 'H2_ppm', 'Ewe/V'], log_y=True)
#fig_ = px.scatter(df_all, x='timestamp', y='Area_H2')
#fig_.show()

In [ ]:
# Save the data in an excel sheet
# excel sheet name:
name_out = 'Pt-C-1-merged-with-all-GC.xlsx'

df_all.to_excel(name_out, sheet_name='FE')

In [ ]:
import xlwings as xw

In [ ]:
wb = xw.Book('Calibrations.xlsx')
sht=wb.sheets[0]
new_wb = xw.Book(name_out)
sht.api.Copy(None, After=new_wb.sheets[-1].api)
wb.close()